In [3]:
!pip install gensim --user

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Looking in indexes: http://rep.msk.mts.ru/artifactory/api/pypi/pypi/simple
     |████████████████████████████████| 24.2MB 29.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 20.0MB/s eta 0:00:01
     |████████████████████████████████| 133kB 21.7MB/s eta 0:00:01
     |████████████████████████████████| 5.9MB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.5MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp27-none-any.whl size=73095 sha256=cc3c4dbf3f9e4a70ed1f67422854bbcccff054d16ed65c305b2051743f73bdf1
  Stored in directory: /home/makilins/.cache/pip/wheels/f9/16/27/09545f427c07feec01

In [1]:
PATH_TO_DATA = 'capstone_user_identification'

In [178]:
import numpy as np
import pandas as pd
import os

pdf = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_data_10users.csv'))
pdf.head()

,session_id,user_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
0,0,1,armmf.adobe,armmf.adobe,armdl.adobe,javadl-esd-secure.oracle,javadl-esd-secure.oracle,oracle,apex.oracle,oracle,gtssl-ocsp.geotrust,gtssl-ocsp.geotrust
1,1,1,gtglobal-ocsp.geotrust,clients1.google,squid-cache.org,geo.mozilla.org,sb-ssl.google,safebrowsing-cache.google,safebrowsing.clients.google,safebrowsing-cache.google,ocsp.thawte,ocsp.verisign
2,2,1,ocsp.digicert,safebrowsing.clients.google,safebrowsing-cache.google,safebrowsing.clients.google,safebrowsing-cache.google,safebrowsing-cache.google,javadl-esd-secure.oracle,javadl-esd-secure.oracle,oracle,oracle
3,3,1,apex.oracle,oracle,oracle,apex.oracle,javadl-esd-secure.oracle,oracle,apex.oracle,javadl-esd-secure.oracle,javadl-esd-secure.oracle,go.microsoft
4,4,1,windows.microsoft,ajax.microsoft,windows.microsoft,res1.windows.microsoft,res2.windows.microsoft,js.microsoft,js.microsoft,res1.windows.microsoft,bing,go.microsoft


In [3]:
corpus = pdf[['site'+str(r) for r in range(1,10+1)]].values.tolist()
corpus[0]

['armmf.adobe',
 'armmf.adobe',
 'armdl.adobe',
 'javadl-esd-secure.oracle',
 'javadl-esd-secure.oracle',
 'oracle',
 'apex.oracle',
 'oracle',
 'gtssl-ocsp.geotrust',
 'gtssl-ocsp.geotrust']

In [6]:
from gensim.models import word2vec

In [25]:
"""
window - длина окна = максимальная длина между текущим словом и предсказываемым 3*2 = 6
workers - распараллеливание
size - размерность выходного вектора
"""

model = word2vec.Word2Vec(corpus, size=300, window=3, workers=4)

In [26]:
"""
model.wv.index2word - список слов в корпусе
model.wv.syn0 - список векторов
"""

w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/opt/Anaconda3-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [27]:
# для каждого слова посчитаем средний вектор

w2v_avg = {}

for w in w2v:
    w2v_avg[w] = np.mean(w2v[w])
    

w2v_avg['9zouxfza1h.s.ad6media.fr']

-0.0016364098

In [15]:
w2v_avg['armmf.adobe']

0.0010619363

In [179]:
for col in ['site'+str(r) for r in range(1,10+1)]:
    pdf[col] = pdf[col].map(w2v)
    

pdf.head()

,session_id,user_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
0,0,1,"[-0.4749068, -0.4874621, -0.46100643, -0.21868...","[-0.4749068, -0.4874621, -0.46100643, -0.21868...","[-0.2537513, -0.15559852, -0.12163722, -0.0412...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.49214545, -0.032901812, 0.008897057, 0.054...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.30773747, -0.109468415, -0.2506138, -0.202...","[-0.30773747, -0.109468415, -0.2506138, -0.202..."
1,1,1,"[-0.22633971, -0.0928885, -0.24093513, -0.1381...","[-0.13731904, -0.03953212, -0.27025098, -0.152...","[-0.062134184, -0.09373012, -0.12133034, 0.027...","[-0.06960947, -0.05322903, -0.07262034, -0.002...","[-0.041435927, -0.042279895, -0.043100335, -0....","[-0.088824645, -0.14135845, -0.011649601, 0.17...","[-0.08384353, -0.124307275, 0.0059213117, 0.09...","[-0.088824645, -0.14135845, -0.011649601, 0.17...","[-0.16387583, -0.11379735, -0.16969353, -0.126...","[-0.17193182, -0.11082508, -0.05160196, -0.082..."
2,2,1,"[-0.1404533, -0.07368097, -0.10539132, -0.1424...","[-0.08384353, -0.124307275, 0.0059213117, 0.09...","[-0.088824645, -0.14135845, -0.011649601, 0.17...","[-0.08384353, -0.124307275, 0.0059213117, 0.09...","[-0.088824645, -0.14135845, -0.011649601, 0.17...","[-0.088824645, -0.14135845, -0.011649601, 0.17...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.8556469, 0.028368898, 0.08195255, 0.158786..."
3,3,1,"[-0.49214545, -0.032901812, 0.008897057, 0.054...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.49214545, -0.032901812, 0.008897057, 0.054...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.8556469, 0.028368898, 0.08195255, 0.158786...","[-0.49214545, -0.032901812, 0.008897057, 0.054...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.5150304, -0.24783584, -0.22485872, -0.1471...","[-0.38584483, -0.0960047, -0.24836135, 0.01628..."
4,4,1,"[-0.27813432, -0.053093437, -0.16285785, 0.101...","[-0.200469, -0.051603667, -0.101219594, 0.0933...","[-0.27813432, -0.053093437, -0.16285785, 0.101...","[-0.26544327, -0.056303106, -0.16320175, 0.129...","[-0.28161076, -0.051825073, -0.12448629, 0.117...","[-0.2670824, -0.033423018, -0.14920586, 0.1031...","[-0.2670824, -0.033423018, -0.14920586, 0.1031...","[-0.26544327, -0.056303106, -0.16320175, 0.129...","[-0.22977471, -0.18269482, -0.28324145, 0.3318...","[-0.38584483, -0.0960047, -0.24836135, 0.01628..."


In [180]:
pdf = pdf.fillna(0)   

In [99]:
s = filter(lambda r: type(r) == list, [[1,2,3], 0])
s

[[1, 2, 3]]

In [13]:
a = [1, 2]
b = [3, 4]

from itertools import izip_longest, imap

for t in zip(a, b):
    print(t)

(1, 3)
(2, 4)


In [58]:
for t in izip_longest(*[a,b]):
    print(t)

(1, 3)
(2, 4)


In [181]:
import numpy

def avg_vector(row):
    sites = filter(lambda r: type(r) == numpy.ndarray, [row['site'+str(r)] for r in range(1,10+1)])

    def avg(x):
        x = [i for i in x if i is not None]
        return sum(x, 0.0) / len(x)
    
    return map(avg, zip(*sites))

In [134]:
row = {
    'site1': [1,2],
    'site2': 0,
    'site3': [10,40]
}
avg_vector(row)

[[1, 2], 0]

In [144]:
pdf.loc[23]

session_id                                                     23
user_id                                                         1
site1           [-0.41874257, -0.21255241, -0.07673472, 0.1211...
site2           [-0.41874257, -0.21255241, -0.07673472, 0.1211...
site3           [0.10105563, -0.09243709, -0.18856826, 0.06079...
site4           [-0.043271843, -0.049936265, -0.026094515, 0.0...
site5           [-0.04680011, -0.09141308, -0.09010201, 0.0834...
site6           [-0.045348644, -0.08613527, -0.069464706, 0.09...
site7           [-0.03874962, -0.060218673, -0.046243243, 0.06...
site8           [-0.03497495, -0.11444219, -0.12921529, 0.1226...
site9           [-0.03630032, -0.06636506, -0.044521198, 0.067...
site10                                                          0
sites_vector                                                   []
Name: 23, dtype: object

In [182]:
pdf['sites_vector'] = pdf.apply(avg_vector, axis = 1)
pdf['sites_vector'].head()

0    [-0.5052539885044098, -0.1821295265108347, -0....
1    [-0.11341388076543808, -0.09533062689006329, -...
2    [-0.3315968871116638, -0.11853047609329223, -0...
3    [-0.59743130505085, -0.08531109802424908, -0.0...
4    [-0.2719019249081612, -0.06677673868834973, -0...
Name: sites_vector, dtype: object

In [183]:
pdf[['vec'+str(r) for r in range(1,301)]] = pd.DataFrame(pdf.sites_vector.values.tolist(), index= pdf.index)
pdf = pdf.fillna(0) 
pdf[['vec'+str(r) for r in range(1,301)]].head()

,vec1,vec2,vec3,vec4,vec5,vec6,vec7,vec8,vec9,vec10,...,vec291,vec292,vec293,vec294,vec295,vec296,vec297,vec298,vec299,vec300
0,-0.505254,-0.182130,-0.182179,-0.080517,0.131558,-0.222450,0.184205,-0.081785,-0.048301,-0.417694,...,-0.121282,-0.078220,-0.200078,-0.199189,-0.377249,-0.283721,-0.404355,-0.185296,-0.134085,0.386315
1,-0.113414,-0.095331,-0.098691,-0.003030,0.164388,0.039337,-0.039631,-0.287422,0.184435,-0.295750,...,-0.103980,-0.248182,-0.244793,-0.249206,-0.249003,-0.200706,-0.071784,-0.092761,-0.046702,0.042452
2,-0.331597,-0.118530,-0.041431,0.059621,0.179973,-0.113783,0.058469,-0.280090,0.135843,-0.458282,...,-0.288484,-0.225221,-0.407367,-0.409702,-0.239298,-0.223382,-0.223002,-0.164918,-0.222416,0.168206
3,-0.597431,-0.085311,-0.065039,0.021580,0.004428,-0.291394,0.276109,-0.034920,-0.070244,-0.398463,...,-0.260878,-0.030368,-0.336718,-0.279815,-0.248160,-0.154243,-0.367584,-0.223831,-0.115881,0.390873
4,-0.271902,-0.066777,-0.170784,0.122584,0.086368,-0.320087,0.237362,-0.003428,-0.254475,-0.347864,...,-0.314552,0.035513,-0.213958,-0.005086,-0.200753,0.048327,-0.046054,-0.287842,0.107349,0.017630


In [184]:
X_10users, y_10users = pdf[['vec'+str(r) for r in range(1,301)]].values, \
                       pdf['user_id'].values

In [170]:
X_10users.shape

(14061, 10)

In [176]:
X_10users.shape

(14061, 300)

In [158]:
y_10users

array([1, 1, 1, ..., 5, 5, 5])

In [185]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

X_train, X_valid, y_train, y_valid = train_test_split(X_10users, y_10users, 
                                                      test_size=0.3, 
                                                     random_state=17, stratify=y_10users)

logit = LogisticRegression(random_state=17)
scores = cross_val_score(logit, X_train, y_train, cv=skf, scoring='accuracy')

print(scores)
print(np.mean(scores))

/home/makilins/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[0.57016743 0.55657213 0.57687614]
0.5678718991133253


In [186]:
%%time

logit_c_values1 = np.logspace(-4, 2, 10)

logit_grid_searcher1 = LogisticRegressionCV(Cs=logit_c_values1,
                                            cv=skf,
                                            multi_class='multinomial',
                                            n_jobs=-1)
logit_grid_searcher1.fit(X_train, y_train)

print(logit_grid_searcher1.scores_)

{128: array([[0.26392694, 0.41552511, 0.44474886, 0.49101979, 0.52328767,
        0.55372907, 0.58569254, 0.60821918, 0.62678843, 0.64048706],
       [0.26471485, 0.42360476, 0.44373285, 0.48490393, 0.50808173,
        0.5336993 , 0.56877097, 0.58828911, 0.61177188, 0.61390668],
       [0.26082977, 0.42220866, 0.45149481, 0.49603417, 0.52135448,
        0.54850519, 0.58419768, 0.61134838, 0.633313  , 0.63849908]]), 1: array([[0.26392694, 0.41552511, 0.44474886, 0.49101979, 0.52328767,
        0.55372907, 0.58569254, 0.60821918, 0.62678843, 0.64048706],
       [0.26471485, 0.42360476, 0.44373285, 0.48490393, 0.50808173,
        0.5336993 , 0.56877097, 0.58828911, 0.61177188, 0.61390668],
       [0.26082977, 0.42220866, 0.45149481, 0.49603417, 0.52135448,
        0.54850519, 0.58419768, 0.61134838, 0.633313  , 0.63849908]]), 5: array([[0.26392694, 0.41552511, 0.44474886, 0.49101979, 0.52328767,
        0.55372907, 0.58569254, 0.60821918, 0.62678843, 0.64048706],
       [0.26471485, 0.423

/home/makilins/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [187]:
for res in logit_grid_searcher1.scores_.items():
    logit_mean_cv_scores1 = res[1][0]

logit_mean_cv_scores1

array([0.26392694, 0.41552511, 0.44474886, 0.49101979, 0.52328767,
       0.55372907, 0.58569254, 0.60821918, 0.62678843, 0.64048706])